In [2]:
import numpy as np
import matplotlib.pyplot as plt
from IPython import display
from sklearn.metrics import mean_squared_error as mse

In [3]:
def awgn(s,SNRdB,L=1):
    """
    AWGN channel
    Add AWGN noise to input signal. The function adds AWGN noise vector to signal 's' to generate a resulting signal vector 'r' of specified SNR in dB. It also
    returns the noise vector 'n' that is added to the signal 's' and the power spectral density N0 of noise added
    Parameters:
        s : input/transmitted signal vector
        SNRdB : desired signal to noise ratio (expressed in dB) for the received signal
        L : oversampling factor (applicable for waveform simulation) default L = 1.
    Returns:
        r : received signal vector (r=s+n)
"""
    gamma = 10**(SNRdB/10) #SNR to linear scale
    if s.ndim==1:# if s is single dimensional vector
        P=L*sum(abs(s)**2)/len(s) #Actual power in the vector
    else: # multi-dimensional signals like MFSK
        P=L*sum(sum(abs(s)**2))/len(s) # if s is a matrix [MxN]
    N0=P/gamma # Find the noise spectral density
    if np.isrealobj(s):# check if input is real/complex object type
        n = np.sqrt(N0/2)*np.random.standard_normal(s.shape) # computed noise
    else:
        n = np.sqrt(N0/2)*(np.random.standard_normal(s.shape)+1j*np.random.standard_normal(s.shape))
    r = s + n # received signal
    return r

In [4]:
def rqnn(y, delta_x, delta_t, X, gamma=2, m=0.1, beta=0.08, zeta=480):
    y = np.array(y)
    time_steps = y.shape[0]
    N = X.shape[0]
    
    t = np.array( [i*delta_t for i in range(time_steps)] )
    psi = np.zeros((N, time_steps))
    psi[:,0:1] = (np.array([np.multiply(np.exp(-np.divide(np.square(X),2)) , 1/np.sqrt(2*np.pi))]).T)
    ipsi = np.zeros((N, time_steps)) + 1j*0
    psi = (psi + ipsi)
    
    y_hat = np.zeros(time_steps)
    
    K = np.zeros((N,time_steps))
    K[:,0] = np.multiply(0.5,(2 * np.random.rand(N) - 1))
    
    v = np.zeros((N,time_steps))
    
    for i in range(time_steps-1):
        for ga in range(gamma):
            inte = 0
            for k1 in range(1,N):
                inte = inte + (X[k1]*(np.linalg.norm(psi[k1,i])**2)*delta_x)
            y_hat[i+1] = inte
            nu = y[i] - y_hat[i]

    #        nu = y[0,i] - y_hat[0,i]
            v[:,i] =-(zeta*nu*K[:,i])
            for k in range(1,N-1):
                psi[k,i+1] = psi[k,i] -1j*delta_t*v[k,i]*psi[k,i] + (1j*delta_t*(psi[k+1,i] - 2*psi[k,i] + psi[k-1,i])/(2*m*delta_x*delta_x) )

            psi[0,i+1] = psi[0,i] -1j*delta_t*v[k,i]*psi[k,i] + (1j*delta_t*(psi[1,i] - 2*psi[0,i]+ psi[N-1,i])/(2*m*delta_x*delta_x) )
            psi[N-1,i+1] = psi[N-1,i] -1j*delta_t*v[k,i]*psi[N-1,i] + (1j*delta_t*(psi[0,i]-2*psi[N-1,i]+psi[N-2,i])/(2*m*delta_x*delta_x) )

            psi[:,i+1] = psi[:,i+1]/np.linalg.norm(psi[:,i+1])
            for k2 in range(1,N):
                K[k2,i+1] = K[k2,i] + beta*nu*(np.linalg.norm(psi[k2,i+1])**2)

        #plot
#         plt.figure(figsize=(8, 6), dpi= 100)

        #with plt.ion():

#         plot1, = plt.plot(t,y,'r',linewidth = 0.7)
#         plt.ylim(-5, 5)
#         plot2, = plt.plot(t,y_hat,linewidth = 0.7);
#         plt.ylim(-5, 5)
#         plt.legend([plot1,plot2],['DC Signal(6db SNR)','Estimated Signal'])
#         display.clear_output(wait=True)
#         plt.pause(0.0001)
        
    return y_hat, psi, v,mse(y,y_hat)

In [5]:
# def step(x):
#     return np.piecewise(x,x>0,(x>=50)&(x<=100),[lambda x:x,lambda x:x])
# x = np.array([-2,2])
# y = awgn(y,6,L=1)

# X = np.arange(-40,40,0.1)
# y_hat, psi, v = rqnn(y, 0.1, 0.01, X)

In [6]:
# def step(x):
#     return np.piecewise(x,[x>=0,(x>=25)&(x<=50),(x>=50)&(x<=75),(x>=75)&(x<=100)],[lambda x:0,lambda x:0.6,lambda x:0,lambda x:0.6])
# delta_t = 0.01
# time_steps = 10000
# t = np.array( [i*delta_t for i in range(time_steps)] )
# y = step(t)
# y = awgn(y,6,L=1)
# X = np.arange(-40,40,0.1)
# y_hat, psi, v = rqnn(y, 0.1, 0.01, X)

In [ ]:

y = awgn(np.zeros(10000)+2,6,L=1)
XX = np.arange(-20,20,0.1)
def f(xx,yy):
    n = len(xx)
    res = []
    for i in range(n):
        res.append(g(xx[i],yy[i]))
    return np.array(res)
        
def g(xx,yy):
    # Assuming rqnn function gives out the mse.
    global y, XX
    _,_,_,er = rqnn(y,0.1, 0.01, XX, beta=xx, zeta=yy)
    return er

c1 = c2 = 0.1
w = 0.8
n_particles = 20
np.random.seed(100)

beta = np.random.rand(n_particles)+0.5
zeta = 100*np.random.rand(n_particles) + 50

beta1 = np.random.randn(n_particles)*0.02
zeta1 = 100*np.random.randn(n_particles)*0.02 


X = np.array([beta,zeta])
V = np.array([beta1,zeta1])
# Initialize data
pbest = X
pbest_obj = f(X[0], X[1])
gbest = pbest[:, pbest_obj.argmin()]
gbest_obj = pbest_obj.min()

print("Initial Parameters")
print(gbest)
print(gbest_obj)
print()
 
def update():
    "Function to do one iteration of particle swarm optimization"
    global V, X, pbest, pbest_obj, gbest, gbest_obj
    # Update params
    r1, r2 = np.random.rand(2)
    V = w * V + c1*r1*(pbest - X) + c2*r2*(gbest.reshape(-1,1)-X)
    X = X + V
    obj = f(X[0], X[1])
    pbest[:, (pbest_obj >= obj)] = X[:, (pbest_obj >= obj)]
    pbest_obj = np.array([pbest_obj, obj]).min(axis=0)
    gbest = pbest[:, pbest_obj.argmin()]
    gbest_obj = pbest_obj.min()

epochs = 23
for j in range(epochs):
    print("Epoch: ", j)
    update()
    print(gbest)
    print(gbest_obj)
    print()

print("********************************** PSO Complete ***********************************")


Initial Parameters
[  1.39132195 109.88433769]
9.951811303231061

Epoch:  0
[  1.39132195 109.88433769]
9.951811303231061

Epoch:  1
[ 1.3511366  90.34671035]
1.7295592625340506

Epoch:  2
